# Machine learning approach for chat bot

### Use case

The goal is to train a machine learning algorithm to be able to advice a user on privacy settings in a specific platform.

In [11]:
import json

with open('training-data.json') as f:
    settings = json.load(f)

print(settings)

{'visibility': {'public_profile': {'description': 'Allows others to view your profile.', 'recommended_action': 'Enable to let others find you.', 'actions': ['Enable to publish your profile.', 'Disable to make your profile not viewable.']}, 'connect_with_me': {'description': 'Allows others to send you connection requests.', 'recommended_action': 'Enable to let others connect with you.', 'actions': ['Enable to allow connection requests.', 'Disable to block connection requests.']}, 'visible_for_search': {'description': 'Controls whether your profile shows up in search results.', 'recommended_action': 'Enable to let others find you.', 'actions': ['Enable to show up in search results.', 'Disable to not show up in search results.']}, 'who_can_find_you': {'description': 'Controls who can find your profile.', 'recommended_action': "Set to 'Logged on only' to only allow platform users to find you.", 'actions': ["Set to 'Everyone' to let everyone find you.", "Set to 'Logged on only' to only allo

In [12]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random


# Load the model
nlp = spacy.load("en_core_web_sm")

# Define the training data
train_data = [
    (settings["visibility"]["public_profile"]["description"], {"cats": {"positive": 1, "negative": 0}}),
    (settings["visibility"]["connect_with_me"]["description"], {"cats": {"positive": 0, "negative": 1}})
]

# Add the textcat component to the pipeline and configure it with the labels
textcat = nlp.add_pipe("textcat")
textcat.add_label("positive")
textcat.add_label("negative")

# Train the model
nlp.begin_training()
for i in range(10):
    for text, annotations in train_data:
        example = Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example])


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

Gradio app

In [13]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        doc = nlp(user_message)
        positive_label = doc.cats['positive'] > doc.cats['negative']
        history[-1][1] = "Positive" if positive_label else "Negative"
        time.sleep(1)
        return history

    # def bot(history):
    #     bot_message = random.choice(["Yes", "No"])
    #     history[-1][1] = bot_message
    #     time.sleep(1)
    #     return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()


AttributeError: 'dict' object has no attribute 'then'